# This notebook generates data plotted in Fig. 11

In [ ]:
import ipyparallel as ipp
clients = ipp.Client()

In [ ]:
clients.ids

In [ ]:
dview = clients.direct_view()

In [ ]:
with dview.sync_imports():
    import numpy as np
    import math
    #from math import exp,sqrt,log
    import numpy.random
    from scipy import signal
%px np = numpy
%px rand = numpy.random

In [ ]:
def Sim_DecMak(trials,randSeed):
    
    rand.seed(randSeed)
    
    def Vin(t):   # square wave
        Vin = np.zeros(2)
        if freq_cond == 0:
            y1 = mag1*signal.square(2 * np.pi * freq1 * t, duty = 0.5)
        else:
            y1 = mag1*signal.square(2 * np.pi * freq1 * t, duty = 0.5*freq1/freq2)
        y2 = mag2*signal.square(2 * np.pi * freq2 * t, duty = 0.5)
        if y1 < 0.2:
            y1 = 0.2
        if y2 < 0.2:
            y2 = 0.2
        Vin[0] = y1
        Vin[1] = y2
        return Vin

    def sampled_stimulus(val1, val2):
        sampledStim = np.zeros(2)
        v1 = val1 + rand.normal(0, standard_dev_phys_stim)
        v2 = val2 + rand.normal(0, standard_dev_phys_stim)
        if v1 < 0.1:
            v1 = 0.1
        if v2 < 0.1:
            v2 = 0.1
        sampledStim[0] = pow(v1, gamma)
        sampledStim[1] = pow(v2, gamma)
        return sampledStim

    def Fdet_mDDM(y, Vin):    # as this notebook produces C-IM data, DDM is not time dependent here
        Fdet = np.zeros(1)
        In1 = Vin[0]
        In2 = Vin[1]
        Fdet[0] = q*(In1-In2) + drift_var_trial
        return Fdet  # 1D array

    def Frand_mDDM(stdDev):
        Frand = np.zeros(1)
        xRand0 = rand.normal(0,1)
        Frand[0] = stdDev*xRand0
        return Frand  # 1D array

    def Fdet_Cross(y, Vin):
        #linear cross-inhibition model time dependent:
        Fdet = np.zeros(2)
        Fdet[0] = -k*y[0]  - beta*y[1] + q*Vin[0]  # to become feedforward
        Fdet[1] = -k*y[1]  - beta*y[0] + q*Vin[1]  # beta is w in some literature
        return Fdet

    def Frand_Cross(stdDev):
        Frand = np.zeros(2)
        xRand0 = rand.normal(0,1)
        xRand1 = rand.normal(0,1)
        Frand[0] = stdDev*xRand0
        Frand[1] = stdDev*xRand1
        return Frand
    def EulerRand(dt, y, Fdet, Frand, Vin, stdDev):
        fd = Fdet(y, Vin)
        fr = Frand(stdDev)
        yNew = y + dt*fd + fr*math.sqrt(dt)
        return yNew
    
    standard_dev_phys_stim = 0.05
    gamma = 0.5
    # model parameters for linear cross inhibition
    k = 0.5 # leak, originally = 0.8
    q = 1 # conversion factor,  originally = 1
    beta = 0.25 # inhibition strength, originally =2

    delps = 0  #  phase shift between inputs
    #freq1 = freq1in #5  # frequencies of signal 1 and 2
    #freq2 = freq2in
    #mag1 = mag1in   # magnitudes of the signals
    #mag2 = mag2in
    #signal = np.cos(omega*time) + np.sin(2*omega*time)
    noiseCoef = 0.1
    sigma = 0.1   # originally = 0.25
    
    tmax = 15
    yinit = np.array([0.0, 0.0])  #initial conditions,  y is 2D array, y relates to LCA
    xinit = np.array([0.0])       # x relates to mDDM data
    ht = 0.001  #time step
    threshold = 0.6
    
    ################################################################################################
    #freq_cond = 0
    
    Opt1_cross = 0
    Opt2_cross = 0
    Undec_cross = 0
    Opt1_mddm = 0
    Opt2_mddm = 0
    Undec_mddm = 0
    DT_cross = []
    DT_mddm = []
    choice_cross = []
    choice_mddm = []
    
    for nn in range(trials):
        t = 0
        y = yinit + rand.uniform(0,0.2,2)
        Input = Vin(t)
        InTransform = sampled_stimulus(Input[0], Input[1])
        while t < tmax and y[0] < 2.0*threshold and y[1] < 2.0*threshold:  
            stdDev = sigma+0.2
            y = EulerRand(ht, y, Fdet_Cross, Frand_Cross, InTransform, stdDev) 
            for j1 in range(len(y)):
                y[j1] = max(0, y[j1])
            t = t + ht
            Input = Vin(t)
            InTransform = sampled_stimulus(Input[0], Input[1])
        if t < tmax:
            DT_cross.append(t)
            if y[0] > y[1]:
                Opt1_cross += 1
                choice_cross.append(1)
            elif y[1] > y[0]:
                Opt2_cross += 1
                choice_cross.append(2)
        else:
            DT_cross.append(100)
            choice_cross.append(100)
            Undec_cross += 1
        
        drift_var_trial = rand.normal(0,0.1)
        t = 0
        x = xinit + rand.uniform(-0.1,0.1)
        Input = Vin(t)
        InTransform = sampled_stimulus(Input[0], Input[1])
        while t < tmax and abs(x) < threshold: 
            variance_tot = (sigma*sigma) + noiseCoef*(InTransform[0]**2 + InTransform[1]**2)
            stdDev = math.sqrt(variance_tot) 
            x = EulerRand(ht, x, Fdet_mDDM, Frand_mDDM, InTransform, stdDev)
            t = t + ht
            Input = Vin(t)
            InTransform = sampled_stimulus(Input[0], Input[1])
        if t < tmax:
            DT_mddm.append(t)
            if x > 0:
                Opt1_mddm += 1
                choice_mddm.append(1)
            elif x < 0:
                Opt2_mddm += 1
                choice_mddm.append(2)
        else:
            DT_mddm.append(100)
            choice_mddm.append(100)
            Undec_mddm += 1
        
    return [mag1,mag2,freq1,freq2], np.mean(np.asarray(DT_cross)), np.std(np.asarray(DT_cross)), Opt1_cross/trials,\
Opt2_cross/trials, Undec_cross/trials, np.mean(np.asarray(DT_mddm)), np.std(np.asarray(DT_mddm)), Opt1_mddm/trials,\
Opt2_mddm/trials, Undec_mddm/trials, DT_cross, DT_mddm, choice_cross, choice_mddm

In [ ]:
import timeit
time_start = timeit.default_timer()
nr_engines = len(clients.ids)
nr_trials_tot = 100000 #2000, 100 fro testing
nr_trials_per_engine = int(nr_trials_tot/nr_engines)
#fref = 5
#freq1list = [3/4*fref,fref,4/3*fref]
#freq2list = [fref]
#mref = 2
#mag1list = [3/4*mref,mref,4/3*mref] 
#mag2list = [mref] 
rho_m_list = [1, 4/3]
rho_freq_list = [1, 3/2]
mean_m_list = [1, 2]
mean_f_list = [2, 4]

freq_cond = 1 # 0 or 1
dview.push(dict(freq_cond = freq_cond))

for rhom in rho_m_list:
    for rhof in rho_freq_list:
        for mean_m in mean_m_list:
            for mean_f in mean_f_list:
                mag1 = 2.0*rhom/(1.0+rhom)*mean_m
                mag2 = 2.0/(1.0+rhom)*mean_m
                freq1 = 2.0*rhof/(1.0+rhof)*mean_f
                freq2 = 2.0/(1.0+rhof)*mean_f
                
                filepath = "DistributionSumData_Freq1_"+str(round(freq1, 2))+"_Freq2_"+str(round(freq2, 2))+"_Mag1_"+str(round(mag1, 2))+"_Mag2_"+str(round(mag2, 2))+"_FreqCond_"+str(freq_cond)+".csv"
                with open(str(filepath), "w") as out:
                    out.write("#meanDTcross, stdDTcross, Opt1_cross, Opt2_cross, Undec_cross, meanDTmddm, stdDTmddm, Opt1_mddm, Opt2_mdd, Undec_mddm\n")
                meanDTcross = 0
                stdDTcross = 0
                Opt1cross = 0
                Opt2cross = 0
                Undec_cross = 0
                meanDTmddm = 0
                stdDTmddm = 0
                Opt1mddm = 0
                Opt2mddm = 0
                Undec_mddm = 0
                DT_cross = []
                DT_mddm = []
                choice_cross = []
                choice_mddm = []
                
                dview.push(dict(freq1 = freq1, freq2 = freq2, mag1 = mag1, mag2 = mag2))
                randSeedList = [(jj+1)*1928374 for jj in range(nr_engines)]
                results = dview.map_sync(Sim_DecMak, [nr_trials_per_engine]*nr_engines, randSeedList)
                for kk in range(nr_engines):
                    meanDTcross += results[kk][1]/nr_engines
                    stdDTcross += results[kk][2]/nr_engines
                    Opt1cross += results[kk][3]/nr_engines
                    Opt2cross += results[kk][4]/nr_engines
                    Undec_cross += results[kk][5]/nr_engines
                    meanDTmddm += results[kk][6]/nr_engines
                    stdDTmddm += results[kk][7]/nr_engines
                    Opt1mddm += results[kk][8]/nr_engines
                    Opt2mddm += results[kk][9]/nr_engines
                    Undec_mddm += results[kk][10]/nr_engines
                    DT_cross += results[kk][11]
                    DT_mddm += results[kk][12]
                    choice_cross += results[kk][13]
                    choice_mddm += results[kk][14]
                with open(str(filepath), "a") as out:
                    out.write(str(meanDTcross)+','+str(stdDTcross)+','+str(Opt1cross)+','+str(Opt2cross)+','+str(Undec_cross)+','+str(meanDTmddm)+','+str(stdDTmddm)+','+str(Opt1mddm)+','+str(Opt2mddm)+','+str(Undec_mddm)+'\n')

                with open("DistributionDTData_Freq1_"+str(round(freq1, 2))+"_Freq2_"+str(round(freq2, 2))+"_Mag1_"+str(round(mag1, 2))+"_Mag2_"+str(round(mag2, 2))+"_FreqCond_"+str(freq_cond)+".csv", "w") as out:
                    out.write("#   ChoiceCross    DTcross   ChoiceMDDM    DTmddm    \n")
                    for jj in range(nr_trials_tot):
                        out.write(str(choice_cross[jj])+','+str(DT_cross[jj])+','+str(choice_mddm[jj])+','+str(DT_mddm[jj])+'\n')
                        
time_stop = timeit.default_timer()
print('elapsed time =',time_stop - time_start,'seconds')